In [6]:
!pip install -q git+https://github.com/srush/MiniChain
!git clone https://github.com/srush/MiniChain; cp -fr MiniChain/examples/* . 

Cloning into 'MiniChain'...
remote: Enumerating objects: 1556, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 1556 (delta 28), reused 25 (delta 25), pack-reused 1518
Receiving objects: 100% (1556/1556), 54.59 MiB | 13.76 MiB/s, done.
Resolving deltas: 100% (982/982), done.


In [7]:
desc = """
### Word Problem Solver

Chain that solves a math word problem by first generating and then running Python code. [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/srush/MiniChain/blob/master/examples/math_demo.ipynb)

(Adapted from Dust [maths-generate-code](https://dust.tt/spolu/a/d12ac33169))
"""

In [8]:

from minichain import show, prompt, OpenAI, Python

In [9]:
@prompt(OpenAI(), template_file="math.pmpt.tpl")
def math_prompt(model, question):
    "Prompt to call GPT with a Jinja template"
    return model(dict(question=question))


In [10]:
@prompt(Python(), template="import math\n{{code}}")
def python(model, code):
    "Prompt to call Python interpreter"
    code = "\n".join(code.strip().split("\n")[1:-1])
    return int(model(dict(code=code)))

In [11]:
def math_demo(question):
    "Chain them together"
    return python(math_prompt(question))

In [12]:

gradio = show(math_demo,
              examples=["What is the sum of the powers of 3 (3^i) that are smaller than 100?",
                        "What is the sum of the 10 first positive integers?",],
                        # "Carla is downloading a 200 GB file. She can download 2 GB/minute, but 40% of the way through the download, the download fails. Then Carla has to restart the download from the beginning. How load did it take her to download the file in minutes?"],
              subprompts=[math_prompt, python],
              out_type="json",
              description=desc,
              )
if __name__ == "__main__":
    gradio.launch()
     

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/Users/bing/opt/anaconda3/envs/web_hw/lib/python3.9/site-packages/gradio/routes.py", line 437, in run_predict
    output = await app.get_blocks().process_api(
  File "/Users/bing/opt/anaconda3/envs/web_hw/lib/python3.9/site-packages/gradio/blocks.py", line 1352, in process_api
    result = await self.call_function(
  File "/Users/bing/opt/anaconda3/envs/web_hw/lib/python3.9/site-packages/gradio/blocks.py", line 1087, in call_function
    raise ValueError("Need to enable queue to use generators.")
ValueError: Need to enable queue to use generators.
Traceback (most recent call last):
  File "/Users/bing/opt/anaconda3/envs/web_hw/lib/python3.9/site-packages/gradio/routes.py", line 437, in run_predict
    output = await app.get_blocks().process_api(
  File "/Users/bing/opt/anaconda3/envs/web_hw/lib/python3.9/site-packages/gradio/blocks.py", line 1352, in process_api
    result = await self.call_function(
  File "/Users/bing/opt/anaconda3/envs/web_

In [13]:

desc = """
### Chat

A chat-like example for multi-turn chat with state. [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/srush/MiniChain/blob/master/examples/chat.ipynb)

(Adapted from [LangChain](https://langchain.readthedocs.io/en/latest/modules/memory/examples/chatgpt_clone.html)'s version of this [blog post](https://www.engraved.blog/building-a-virtual-machine-inside/).)

"""
     

In [14]:
from dataclasses import dataclass, replace
from typing import List, Tuple
from minichain import OpenAI, prompt, show

In [15]:
MEMORY = 10

In [16]:
@dataclass
class State:
    memory: List[Tuple[str, str]]
    human_input: str = ""

    def push(self, response: str) -> "State":
        memory = self.memory if len(self.memory) < MEMORY else self.memory[1:]
        return State(memory + [(self.human_input, response)])

In [17]:

@prompt(OpenAI(), template_file="chat.pmpt.tpl")
def chat_prompt(model, state: State) -> State:
    out = model(state)
    result = out.split("Assistant:")[-1]
    return state.push(result)

In [18]:
examples = [
    "ls ~",
    "cd ~",
    "{Please make a file jokes.txt inside and put some jokes inside}",
    """echo -e "x=lambda y:y*5+3;print('Result:' + str(x(6)))" > run.py && python3 run.py""",
    """echo -e "print(list(filter(lambda x: all(x%d for d in range(2,x)),range(2,3**10)))[:10])" > run.py && python3 run.py""",
    """echo -e "echo 'Hello from Docker" > entrypoint.sh && echo -e "FROM ubuntu:20.04\nCOPY entrypoint.sh entrypoint.sh\nENTRYPOINT [\"/bin/sh\",\"entrypoint.sh\"]">Dockerfile && docker build . -t my_docker_image && docker run -t my_docker_image""",
    "nvidia-smi"
]
     

In [19]:

gradio = show(lambda command, state: chat_prompt(replace(state, human_input=command)),
              initial_state=State([]),
              subprompts=[chat_prompt],
              examples=examples,
              out_type="json",
              description=desc,
)
if __name__ == "__main__":
    gradio.launch()
     

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/Users/bing/opt/anaconda3/envs/web_hw/lib/python3.9/site-packages/gradio/routes.py", line 437, in run_predict
    output = await app.get_blocks().process_api(
  File "/Users/bing/opt/anaconda3/envs/web_hw/lib/python3.9/site-packages/gradio/blocks.py", line 1352, in process_api
    result = await self.call_function(
  File "/Users/bing/opt/anaconda3/envs/web_hw/lib/python3.9/site-packages/gradio/blocks.py", line 1087, in call_function
    raise ValueError("Need to enable queue to use generators.")
ValueError: Need to enable queue to use generators.
